### Download Flowers dataset

In [0]:
#You can download the data manually as well instead of using 'wget'
!wget http://download.tensorflow.org/example_images/flower_photos.tgz --quiet

In [0]:
#Read the dataset
import tarfile
dataset = tarfile.open('flower_photos.tgz')

In [0]:
#We will build a pandas dataset
import pandas as pd
df = pd.DataFrame(columns=['class','image_file'])

In [0]:
#Run through tarfile members 
for name in dataset.getnames():
    
    tar_mem = dataset.getmember(name)
    
    #Check if it is a file
    if(tar_mem.isfile() and name.endswith('.jpg')):
        #Build directory and class info
        im_dir = name[0:name.rfind('/')]
        im_class = im_dir[im_dir.rfind('/')+1:]
        #Add record to the dataframe
        df.loc[df.shape[0]] = [im_class, name]

In [0]:
#extract data
dataset.extractall(path='')

In [0]:
!ls -l flower_photos

total 608
drwx------ 2 270850 5000  36864 Feb 10  2016 daisy
drwx------ 2 270850 5000  49152 Feb 10  2016 dandelion
-rw-r----- 1 270850 5000 418049 Feb  9  2016 LICENSE.txt
drwx------ 2 270850 5000  36864 Feb 10  2016 roses
drwx------ 2 270850 5000  36864 Feb 10  2016 sunflowers
drwx------ 2 270850 5000  40960 Feb 10  2016 tulips


Create Training & Test Dataset

In [0]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [0]:
train_df.to_csv('data/flower_photos/train.csv',index=False)
test_df.to_csv('data/flower_photos/test.csv', index=False)

### Read training and test data

In [0]:
#Read training and test Dataframe
train_df = pd.read_csv('data/flower_photos/train.csv')
test_df = pd.read_csv('data/flower_photos/test.csv')

In [0]:
#Check contents
train_df.sample(n=5)

,class,image_file
69,tulips,flower_photos/tulips/6227136683_262c6be56b.jpg
2189,roses,flower_photos/roses/3560426426_1c66cb8330.jpg
1645,roses,flower_photos/roses/18599603859_f2ec616ddf_n.jpg
987,daisy,flower_photos/daisy/7191221492_610035de7c_m.jpg
141,tulips,flower_photos/tulips/4582198748_20fa7caaa1.jpg


In [0]:
#Get class names
class_names = train_df['class'].unique().tolist()
print('Flower classes: ', class_names)

Flower classes:  ['tulips', 'daisy', 'sunflowers', 'dandelion', 'roses']


### Build Batch generator (using ImageDataGenerator)

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
#Define some parameters
img_size = 224
img_depth = 3  

Function to normalize image according to Model being used

In [0]:
def normalize_data(img):
    
    #Normalize for MobileNet
    return tf.keras.applications.mobilenet.preprocess_input(img)

Defime ImageDataGenerator for both Training and Test Separately

In [0]:
#Define Training Data Generator with augmentations
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.2,
                                                                horizontal_flip=True,
                                                                preprocessing_function=normalize_data) #Normalize the data accordingly

#Define Test Data Generator with NO augmentations
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=normalize_data) #Normalize the data accordingly

Create Data Generators objects for Training and Test

In [0]:
#Training (from dataframe)
train_generator = train_datagen.flow_from_dataframe(train_df, 
                                                    x_col='image_file', #File path for image
                                                    y_col='class',           #Class for the image
                                                    target_size=(img_size, img_size), #Image resize dimensions
                                                    batch_size=64)

Found 2936 validated image filenames belonging to 5 classes.


In [0]:
#Test (from dataframe)
test_generator = test_datagen.flow_from_dataframe(test_df,
                                                  x_col='image_file', #File path for image
                                                  y_col='class',           #Class for the image
                                                  target_size=(img_size, img_size), #Image resize dimensions
                                                  batch_size=64)

Found 734 validated image filenames belonging to 5 classes.


### Custom Batch Generator
This code can replace work done by ImageDataGenerator above

In [0]:
def batch_generator(df, batchsize=32, train_mode=True):   
    
    img_generator= tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                                   width_shift_range=0.2,
                                                                   height_shift_range=0.2,
                                                                   horizontal_flip=True)
    
    while True:       
        
        
        #Generate random numbers to pick images from dataset
        batch_nums = np.random.randint(0,df.shape[0],batchsize)
        
        #Initialize batch images array
        batch_images = np.zeros((batchsize,img_size, img_size,img_depth))
        
        #Initiate batch label array
        batch_labels = np.zeros((batchsize, len(class_names)))
        
        for i in range(batchsize):
            
            #Load image
            flower_image =  tf.keras.preprocessing.image.load_img(df.loc[batch_nums[i]]['image_file'],
                                                                  target_size=(img_size, img_size))
            #Convert to array
            flower_image = tf.keras.preprocessing.image.img_to_array(flower_image)
            
            if(train_mode):
                #Apply transform
                flower_image =  img_generator.random_transform(flower_image)
            
            
            #Get the class
            img_class = df.loc[batch_nums[i]]['class']
            #Conver class to number
            img_class = class_names.index(img_class)
            #Convert class to one hot encoding
            img_class = tf.keras.utils.to_categorical(img_class, num_classes=len(class_names))
            
            #Update batch images and class arrays
            #DO NOT FORGET THIS STEP!!!
            batch_images[i] = tf.keras.applications.mobilenet.preprocess_input(flower_image)
            batch_labels[i] = img_class
        
        
        yield batch_images, batch_labels          

### Load pre-trained model

In [0]:
tf.keras.backend.clear_session()
model = tf.keras.applications.MobileNet(include_top=False, #Do not include FC layer at the end
                                        input_shape=(img_size,img_size, img_depth),
                                        weights='imagenet')

In [0]:
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [0]:
model.output

<tf.Tensor 'conv_pw_13_relu/Relu6:0' shape=(?, 7, 7, 1024) dtype=float32>

Freeze the layers in Pre-trained model

In [0]:
#Set pre-trained model layers to not trainable
for layer in model.layers:
    layer.trainable = False

### Add FC layer for new classes

In [0]:
#get Output layer of Pre0trained model
x = model.output

#Flatten the output to feed to Dense layer
x = tf.keras.layers.GlobalAveragePooling2D()(x)

#Add output layer
prediction = tf.keras.layers.Dense(len(class_names),activation='softmax')(x)

### Building final model for Classification

In [0]:
#Using Keras Model class
final_model = tf.keras.models.Model(inputs=model.input, #Pre-trained model input as input layer
                                    outputs=prediction) #Output layer added

In [0]:
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
final_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128   

### Train the model

In [0]:
final_model.fit_generator(train_generator, 
                          epochs=5,
                          steps_per_epoch= train_df.shape[0]//batchsize,
                          validation_data=test_generator,
                          validation_steps = test_df.shape[0]//batchsize)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
45/45 [==============================] - 50s 1s/step - loss: 1.0820 - acc: 0.5903 - val_loss: 0.8886 - val_acc: 0.6676
Epoch 2/5
45/45 [==============================] - 47s 1s/step - loss: 0.5503 - acc: 0.8073 - val_loss: 0.7338 - val_acc: 0.7486
Epoch 3/5
45/45 [==============================] - 47s 1s/step - loss: 0.4529 - acc: 0.8431 - val_loss: 0.6795 - val_acc: 0.7855
Epoch 4/5
45/45 [==============================] - 47s 1s/step - loss: 0.3783 - acc: 0.8701 - val_loss: 0.7184 - val_acc: 0.7557
Epoch 5/5
45/45 [==============================] - 47s 1s/step - loss: 0.3548 - acc: 0.8760 - val_loss: 0.5180 - val_acc: 0.8182


In [0]:
final_model.fit_generator(train_generator, 
                          epochs=10,
                          steps_per_epoch= train_df.shape[0]//batchsize,
                          validation_data=test_generator,
                          validation_steps = test_df.shape[0]//batchsize, initial_epoch=5)

Epoch 6/10
45/45 [==============================] - 50s 1s/step - loss: 0.3308 - acc: 0.8885 - val_loss: 0.6078 - val_acc: 0.8054
Epoch 7/10
45/45 [==============================] - 47s 1s/step - loss: 0.3095 - acc: 0.8885 - val_loss: 0.6226 - val_acc: 0.7884
Epoch 8/10
45/45 [==============================] - 47s 1s/step - loss: 0.2941 - acc: 0.9003 - val_loss: 0.6213 - val_acc: 0.8111
Epoch 9/10
45/45 [==============================] - 47s 1s/step - loss: 0.2781 - acc: 0.9038 - val_loss: 0.5862 - val_acc: 0.8040
Epoch 10/10
45/45 [==============================] - 47s 1s/step - loss: 0.2705 - acc: 0.9118 - val_loss: 0.6798 - val_acc: 0.7812


In [0]:
print(len(model.layers))

87


### Unfreeze some of Trained Layers in Mobilenet

In [0]:
#Unfreezing all layers after layer# 55
for layer in model.layers[56:]:
    layer.trainable =  True    

In [0]:
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
final_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128   

In [0]:
final_model.fit_generator(train_generator, 
                          epochs=50,
                          steps_per_epoch= train_df.shape[0]//batchsize,
                          validation_data=test_generator,
                          validation_steps = test_df.shape[0]//batchsize)

Epoch 1/50
45/45 [==============================] - 52s 1s/step - loss: 0.4532 - acc: 0.8698 - val_loss: 4.7370 - val_acc: 0.5412
Epoch 2/50
45/45 [==============================] - 48s 1s/step - loss: 0.2198 - acc: 0.9344 - val_loss: 1.7840 - val_acc: 0.7386
Epoch 3/50
45/45 [==============================] - 48s 1s/step - loss: 0.1410 - acc: 0.9552 - val_loss: 3.2912 - val_acc: 0.6548
Epoch 4/50
45/45 [==============================] - 48s 1s/step - loss: 0.1243 - acc: 0.9632 - val_loss: 1.2881 - val_acc: 0.8026
Epoch 5/50
45/45 [==============================] - 48s 1s/step - loss: 0.0724 - acc: 0.9764 - val_loss: 0.7270 - val_acc: 0.8864
Epoch 6/50
45/45 [==============================] - 48s 1s/step - loss: 0.0643 - acc: 0.9785 - val_loss: 0.8096 - val_acc: 0.8651
Epoch 7/50
45/45 [==============================] - 48s 1s/step - loss: 0.0877 - acc: 0.9705 - val_loss: 1.6317 - val_acc: 0.7983
Epoch 8/50
45/45 [==============================] - 48s 1s/step - loss: 0.0797 - acc: 0.97